<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml3/blob/main/term2_sprint21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint21 自然言語処理入門

## 1.このSprintについて

### Sprintの目的
- 自然言語処理の一連の流れを学ぶ
- 自然言語のベクトル化の方法を学ぶ

### どのように学ぶか
自然言語処理定番のデータセットを用いて、一連の流れを見ていきます。

## 2.自然言語のベクトル化
自然言語処理（NLP, Natural Language Processing） とは人間が普段使っている 自然言語 をコンピュータに処理させる技術のことです。ここではその中でも、機械学習の入力として自然言語を用いることを考えていきます。

多くの機械学習手法は 数値データ（量的変数） の入力を前提にしていますので、自然言語の テキストデータ を数値データに変換する必要があります。これを 自然言語のベクトル化 と呼びます。ベクトル化の際にテキストデータの特徴をうまく捉えられるよう、様々な手法が考えられてきていますので、このSprintではそれらを学びます。

### 非構造化データ
データの分類として、表に数値がまとめられたようなコンピュータが扱いやすい形を 構造化データ 、人間が扱いやすい画像・動画・テキスト・音声などを 非構造化データ と呼ぶことがあります。自然言語のベクトル化は、非構造化データを構造化データに変換する工程と言えます。同じ非構造化データでも、画像に対してはディープラーニングを用いる場合この変換作業はあまり必要がありませんでしたが、テキストにおいてはこれをどう行うかが重要です。

### 自然言語処理により何ができるか
機械学習の入力や出力に自然言語のテキストを用いることで様々なことができます。入力も出力もテキストである例としては 機械翻訳 があげられ、実用化されています。入力は画像で出力がテキストである 画像キャプション生成 やその逆の文章からの画像生成も研究が進んでいます。

しかし、出力をテキストや画像のような非構造化データとすることは難易度が高いです。比較的簡単にできることとしては、入力をテキスト、出力をカテゴリーとする テキスト分類 です。

アヤメやタイタニック、手書き数字のような定番の存在として、IMDB映画レビューデータセット の感情分析があります。レビューの文書が映画に対して肯定的か否定的かを2値分類します。文書ごとの肯定・否定はラベルが与えられています。このSprintではこれを使っていきます。

## 3.IMDB映画レビューデータセットの準備
IMDB映画レビューデータセットを準備します。

### ダウンロード
次のwgetコマンドによってダウンロードします。


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/diveintocode-ml/term2/term2_sprintt21')
print(os.getcwd())

In [ ]:
# # IMDBをカレントフォルダにダウンロード
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# # 解凍
# !tar zxf aclImdb_v1.tar.gz
# # aclImdb/train/unsupはラベル無しのため削除
# !rm -rf aclImdb/train/unsup
# # IMDBデータセットの説明を表示
# !cat aclImdb/README

以下のサイトで公開されているデータセットです。


Sentiment Analysis  
http://ai.stanford.edu/~amaas/data/sentiment/ 


読み込み
scikit-learnのload_filesを用いて読み込みます。


sklearn.datasets.load_files — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html


《読み込むコード》

In [ ]:
from sklearn.datasets import load_files
train_review = load_files('aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target
test_review = load_files('aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target
# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

In [ ]:
import numpy as np

print(np.array(x_train).shape)
print(np.array(y_train).shape)
print(np.array(x_test).shape)
print(np.array(y_test).shape)

### このデータセットについて
中身を見てみると、英語の文章が入っていることが分かります。

In [ ]:
print("x : {}".format(x_train[0]))

IMDBはInternet Movie Databaseの略で、映画のデータベースサイトです。


Ratings and Reviews for New Movies and TV Shows - IMDb  
https://www.imdb.com/

このサイトではユーザーが映画に対して1から10点の評価とコメントを投稿することができます。そのデータベースから訓練データは25000件、テストデータは25000件のデータセットを作成しています。


4点以下を否定的、7点以下を肯定的なレビューとして2値のラベル付けしており、これにより感情の分類を行います。5,6点の中立的なレビューはデータセットに含んでいません。また、ラベルは訓練用・テスト用それぞれで均一に入っています。詳細はダウンロードしたREADMEを確認してください。

## 4.古典的な手法

古典的ながら現在でも強力な手法であるBoWとTF-IDFを見ていきます。

## 5.BoW

単純ながら効果的な方法として BoW (Bag of Words) があります。これは、サンプルごとに単語などの 登場回数 を数えたものをベクトルとする方法です。単語をカテゴリとして捉え one-hot表現 していることになります。


### 例
例として、IMDBデータセットからある3文の最初の5単語を抜き出したものを用意しました。



In [ ]:
mini_dataset = \
  ["This movie is very good.",
  "This film is a good",
  "Very bad. Very, very bad."]

### この3文にBoWを適用させてみます。scikit-learnのCountVectorizerを利用します。


sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
# テキストドキュメントのコレクションをトークンカウントのマトリックスに変換
vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()
        #------------------------------
        # 語彙辞書を学び、ドキュメントと用語のマトリックスを返す
# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

例にあげた3文の中で登場する8種類の単語が列名になり、0,1,2番目のサンプルでそれらが何回登場しているかを示しています。2番目のサンプル「Very bad. Very, very bad.」ではbadが2回、veryが3回登場しています。列名になっている言葉はデータセットが持つ 語彙 と呼びます。


テキストはBoWにより各サンプルが語彙数の次元を持つ特徴量となり、機械学習モデルへ入力できるようになります。この時使用したテキスト全体のことを コーパス と呼びます。語彙はコーパスに含まれる言葉よって決まり、それを特徴量としてモデルの学習を行います。そのため、テストデータではじめて登場する語彙はベクトル化される際に無視されます。


### 前処理
CountVectorizerクラスでは大文字は小文字に揃えるという 前処理 が自動的に行われています。こういった前処理は自然言語処理において大切で、不要な記号などの消去（テキストクリーニング）や表記揺れの統一といったことを別途行うことが一般的です。


語形が「see」「saw」「seen」のように変化する単語に対して語幹に揃える ステミング と呼ばれる処理を行うこともあります。


### トークン
BoWは厳密には単語を数えているのではなく、 トークン（token） として定めた固まりを数えます。


何をトークンとするかはCountVectorizerでは引数token_patternで 正規表現 の記法により指定されます。デフォルトはr'(?u)\b\w\w+\b'ですが、上の例ではr'(?u)\b\w+\b'としています。


デフォルトでは空白・句読点・スラッシュなどに囲まれた2文字以上の文字を1つのトークンとして抜き出すようになっているため、「a」や「I」などがカウントされません。英語では1文字の単語は文章の特徴をあまり表さないため、除外されることもあります。しかし、上の例では1文字の単語もトークンとして抜き出すように引数を指定しています。


**《正規表現》**


正規表現は前処理の際にも活用しますが、ここでは詳細は扱いません。Pythonではreモジュールによって正規表現操作ができます。


re — 正規表現操作  
https://docs.python.org/ja/3/library/re.html


正規表現を利用する際はリアルタイムで結果を確認できる以下のようなサービスが便利です。


Online regex tester and debugger: PHP, PCRE, Python, Golang and JavaScript  
https://regex101.com/

### 0.0.1（予備知識）正規表現とは？
https://userweb.mnet.ne.jp/nakama/

### 形態素解析
英語などの多くの言語では空白という分かりやすい基準でトークン化が行えますが、日本語ではそれが行えません。


日本語では名詞や助詞、動詞のように異なる 品詞 で分けられる単位で 分かち書き することになります。例えば「私はプログラミングを学びます」という日本語の文は「私/は/プログラミング/を/学び/ます」という風になります。


これには MeCab や Janome のような形態素解析ツールを用います。Pythonから利用することも可能です。MeCabをウェブ上で簡単に利用できるWeb茶まめというサービスも国立国語研究所が提供しています。


自然言語では新しい言葉も日々生まれますので、それにどれだけ対応できるかも大切です。MeCab用の毎週更新される辞書として mecab-ipadic-NEologd がオープンソースで存在しています。


mecab-ipadic-neologd/README.ja.md at master · neologd/mecab-ipadic-neologd  
https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md


### n-gram
上のBoWの例では1つの単語（トークン）毎の登場回数を数えましたが、これでは語順は全く考慮されていません。


考慮するために、隣あう単語同士をまとめて扱う n-gram という考え方を適用することがあります。2つの単語をまとめる場合は 2-gram (bigram) と呼び、次のようになります。

In [ ]:
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
            #--------------------------------
            # 生の文書のすべてのトークンの語彙辞書を学びます。
                                                  #--------------
                                                  #疎行列（スパース行列）を効率的に扱う。
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

### 0.0.2（予備知識） 疎行列（スパース行列）とは？  
https://qiita.com/KQTS/items/e5500ba6e2681456e268

2-gramにより「very good」と「very bad」が区別して数えられています。


単語をまとめない場合は 1-gram (unigram) と呼びます。3つまとめる3-gram(trigram)など任意の数を考えることができます。1-gramと2-gramを組み合わせてBoWを行うといったこともあります。



### 【問題1】BoWのスクラッチ実装
以下の3文のBoWを求められるプログラムをscikit-learnを使わずに作成してください。1-gramと2-gramで計算してください。

In [ ]:
# 提供データ
mini_dataset = \
  ["This movie is SOOOO funny!!!",
  "What a movie! I never",
  "best movie ever!!!!! this movie"]

1.1.1（解答） 1-gram

In [ ]:
# リスト内の文字を全て小文字に変換
data = list(map(str.lower,mini_dataset))
data

In [ ]:
word_list = [word.split(" ") for word in data]
word_list

In [ ]:
import string
kigo = string.punctuation # >>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
table = str.maketrans( "", "", kigo)
kigo

In [ ]:
# 文字の型を標準化するためのステイミング 
# 注意:機会にとっての処理しやすい文字列に変える技術のため、
# 人が認識できる言語としては意味を成さない場合がある。

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
words = []
split_words = []
for word in word_list:
  a = []
  for moji in word:
    moji = ps.stem(moji)
    split_words.append(moji.translate(table))
    a.append(moji.translate(table))
  words.append(a)
print(words)
print(split_words)

In [ ]:
result_dict1 = {}
for i in split_words:
    one_list = []
    for j in words:
      one_list.append(j.count(i))
      result_dict1[i] = one_list

In [ ]:
result_dict1

In [ ]:
result_df = pd.DataFrame(result_dict1, index=data)
result_df

### 1.2.1（解答） 2-gram

In [ ]:
print(words)
print(split_words)

In [ ]:
words2 = []
split_words2 = []

for l in words:
  b = []
  for a in range(len(l) -2 +1):    
    ab = l[a : a + 2]
    d = ab[0] +" " + ab[1]
    split_words2.append(d)
    b.append(d)
  words2.append(b)

In [ ]:
words2

In [ ]:
split_words2

In [ ]:
result_dict2 = {}
for i in split_words2:
    one_list = []
    for j in words2:
      one_list.append(j.count(i))
      result_dict2[i] = one_list

In [ ]:
result_df = pd.DataFrame(result_dict2, index=data)
result_df

## 6.TF-IDF

BoWの発展的手法として TF-IDF もよく使われます。これは Term Frequency (TF) と Inverse Document Frequency (IDF) という2つの指標の組み合わせです。


**《標準的なTF-IDFの式》**


Term Frequency:


$$tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}$$

$n_{t,d}$ : サンプルd内のトークンtの出現回数（BoWと同じ）


$\sum_{s \in d}n_{s,d}$ : サンプルdの全トークンの出現回数の和


Inverse Document Frequency:


$$idf(t) = \log{\frac{N}{df(t)}}$$
$N$ : サンプル数


$df(t)$ : トークンtが出現するサンプル数


＊logの底は任意の値


TF-IDF:


$$tfidf(t, d) = tf(t, d) \times idf(t)$$


### IDF
IDFはそのトークンがデータセット内で珍しいほど値が大きくなる指標です。


サンプル数 $N$ をIMDB映画レビューデータセットの訓練データに合わせ25000として、トークンが出現するサンプル数 $df(t)$ を変化させたグラフを確認してみると、次のようになります。



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
n_samples = 25000
idf = np.log(n_samples/np.arange(1,n_samples))
plt.title("IDF")
plt.xlabel("df(t)")
plt.ylabel("IDF")
plt.plot(idf)
plt.show()

TF-IDFではこの数を出現回数に掛け合わせるので、珍しいトークンの登場に重み付けを行なっていることになります。


### ストップワード
あまりにも頻繁に登場するトークンは、値を小さくするだけでなく、取り除くという前処理を加えることもあります。取り除くもののことを ストップワード と呼びます。既存のストップワード一覧を利用したり、しきい値によって求めたりします。


scikit-learnのCountVectorizerでは引数stop_wordsにリストで指定することで処理を行なってくれます。

In [ ]:
vectorizer = CountVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

代表的な既存のストップワード一覧としては、NLTK という自然言語処理のライブラリのものがあげられます。あるデータセットにおいては特別重要な意味を持つ単語が一覧に含まれている可能性もあるため、使用する際は中身を確認することが望ましいです。

In [ ]:
# はじめて使う場合はストップワードをダウンロード
import nltk
stop_words = nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print("stop word : {}".format(stop_words)) # 'i', 'me', 'my', ...

逆に、登場回数が特に少ないトークンも取り除くことが多いです。全てのトークンを用いるとベクトルの次元数が著しく大きくなってしまい計算コストが高まるためです。


scikit-learnのCountVectorizerでは引数max_featuresに最大の語彙数を指定することで処理を行なってくれます。以下の例では出現数が多い順に5個でベクトル化しています。

In [ ]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features = 5)
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

## 【問題2】TF-IDFの計算
IMDB映画レビューデータセットをTF-IDFによりベクトル化してください。NLTKのストップワードを利用し、最大の語彙数は5000程度に設定してください。テキストクリーニングやステミングなどの前処理はこの問題では要求しません。


TF-IDFの計算にはscikit-learnの以下のどちらかのクラスを使用してください。


sklearn.feature_extraction.text.TfidfVectorizer — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

sklearn.feature_extraction.text.TfidfTransformer — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html


なお、scikit-learnでは標準的な式とは異なる式が採用されています。


また、デフォルトではnorm="l2"の引数が設定されており、各サンプルにL2正規化が行われます。norm=Noneとすることで正規化は行われなくなります。


Term Frequency:

$$tf(t,d) = n_{t,d}$$


$ntd$ : サンプルd内のトークンtの出現回数


scikit-learnのTFは分母がなくなりBoWと同じ計算になります。


Inverse Document Frequency:

$$idf(t) = \log{\frac{1+N}{1+df(t)}}+1$$

$N$ : サンプル数


$df(t)$ : トークンtが出現するサンプル数


＊logの底はネイピア数e


詳細は以下のドキュメントを確認してください。


5.2.3.4. Tf–idf term weighting — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

### 2.1.1（解答）

In [ ]:
# 生のドキュメントのコレクションをTF-IDF機能のマトリックスに変換します。 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stop_words, max_features = 5000)
X_vec = vectorizer.fit_transform(x_train).toarray()

print(f"X_vec.shape:{X_vec.shape}")

In [ ]:
y_train.shape

## 【問題3】TF-IDFを用いた学習
問題2で求めたベクトルを用いてIMDB映画レビューデータセットの学習・推定を行なってください。モデルは2値分類が行える任意のものを利用してください。


ここでは精度の高さは求めませんが、最大の語彙数やストップワード、n-gramの数を変化させて影響を検証してみてください。

### 3.1.1(解答)ストップワード使用、最大語彙数5000 (問題2の設定)

In [ ]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_vec, y_train, test_size=0.2)
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

In [ ]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, Y_train)
y_predict = clf.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
print("正解率：",accuracy_score(y_predict, Y_val))

### 3.2.1(解答) ストップワードなし、最大語彙数5000

In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000)
X_vec1 = vectorizer.fit_transform(x_train).toarray()

print(f"X_vec1.shape:{X_vec1.shape}")

In [ ]:
X_train1, X_val1, Y_train, Y_val = train_test_split(X_vec1, y_train, test_size=0.2)
print(X_train1.shape, X_val1.shape, Y_train.shape, Y_val.shape)

In [ ]:
clf1 = LogisticRegression()
clf1.fit(X_train1, Y_train)
y_predict1 = clf1.predict(X_val1)
print("正解率：",accuracy_score(y_predict1, Y_val))

### 3.3.1(解答) ストップワードなし、最大語彙数2500

In [ ]:
vectorizer = TfidfVectorizer(max_features = 2500)
X_vec2 = vectorizer.fit_transform(x_train).toarray()

X_train2, X_val2, Y_train, Y_val = train_test_split(X_vec2, y_train, test_size=0.2)
print(X_train2.shape, X_val2.shape, Y_train.shape, Y_val.shape)

In [ ]:
clf2 = LogisticRegression()
clf2.fit(X_train2, Y_train)
y_predict2 = clf2.predict(X_val2)
print("正解率：",accuracy_score(y_predict2, Y_val))

### 3.4.1(解答) ストップワード使用、最大語彙数5000 2-gram

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(2, 2), max_features=5000)
X_vec3 = vectorizer.fit_transform(x_train).toarray()

X_train3, X_val3, Y_train, Y_val = train_test_split(X_vec3, y_train, test_size=0.2)
print(X_train3.shape, X_val3.shape, Y_train.shape, Y_val.shape)

In [ ]:
clf3 = LogisticRegression()
clf3.fit(X_train3, Y_train)
y_predict3 = clf3.predict(X_val3)
print("正解率：",accuracy_score(y_predict3, Y_val))

## 7.Word2Vec

ニューラルネットワークを用いてベクトル化を行う手法が Word2Vec です。


BoWやTF-IDFはone-hot表現であったため、得られるベクトルの次元は語彙数分になります。そのため、語彙数を増やしにくいという問題があります。一方で、Word2Vecでは単語を任意の次元のベクトルに変換します。これをを Word Embedding（単語埋め込み） や 分散表現 と呼びます。変換操作を「ベクトル空間に埋め込む」と言うことが多いです。


Word2VecにはCBoWとSkip-gramという2種類の仕組みがあるため順番に見ていきます。


### CBoW
CBoW (Continuous Bag-of-Words) によるWord2Vecではある単語とある単語の間に来る単語を推定できるように全結合層2層のニューラルネットワークを学習します。


単語はコーパスの語彙数次元のone-hot表現を行なっておきます。そのため、入力と出力の次元は語彙数と同じになります。一方で、中間のノード数をWord2Vecにより得たい任意の次元数とします。これにより全結合層の重みは「得たい次元のノード数×語彙数」になります。このネットワークにより学習を行なった後、出力側の重みを取り出すことで、各語彙を表すベクトルを手に入れることができます。


間の単語の推定を行なっているため、同じ箇所で代替可能な言葉は似たベクトルになるというメリットもあります。これはBoWやTF-IDFでは得られない情報です。


あるテキストは「そのテキストの長さ（単語数）×Word2Vecで得た分散表現の次元数」の配列になりますが、各入力の配列を揃える必要があるモデルに入力するためには、短いテキストは空白を表す単語を加える パディング を行なったり、長いテキストは単語を消したりします。テキストを 固定長 にすると呼びます。


### ウィンドウサイズ
入力する単語は推定する前後1つずつだけでなく、複数個とする場合もあります。前後いくつを見るかの大きさを ウィンドウサイズ と呼びます。


### Skip-gram
CBoWとは逆にある単語の前後の単語を推定できるように全結合層2層のニューラルネットワークを学習する方法が Skip-gram です。学習を行なった後は入力側の重みを取り出し各語彙を表すベクトルとします。現在一般的に使われているのはCBoWよりもSki-gramです。


### 利用方法
Pythonでは Gensim ライブラリを用いて扱うことができます。


gensim: models.word2vec – Word2vec embeddings  
https://radimrehurek.com/gensim/models/word2vec.html


BoWの例と同じ文章で学習してみます。CountVectorizerと異なり前処理を自動的に行なってはくれないため、単語（トークン）はリストで分割しておきます。また、大文字は小文字に揃え、記号は取り除きます。


デフォルトのパラメータではCBoWで計算されます。また、ウィンドウサイズはwindow=5に設定されています。

In [ ]:
from gensim.models import Word2Vec
sentences = [['this', 'movie', 'is', 'very', 'good'], ['this', 'film', 'is', 'a', 'good'], ['very', 'bad', 'very', 'very', 'bad']]
model = Word2Vec(min_count=1, size=10) # 次元数を10に設定
model.build_vocab(sentences) # 準備
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter) # 学習
print("語彙の一覧 : {}".format(model.wv.vocab.keys()))
for vocab in model.wv.vocab.keys():
  print("{}のベクトル : \n{}".format(vocab, model.wv[vocab]))

このようにしてベクトルが得られます。


### 単語の距離
ベクトル間で計算を行うことで、ある単語に似たベクトルを持つ単語を見つけることができます。例えばgoodに似たベクトルの単語を3つ探します。

In [ ]:
model.wv.most_similar(positive="good", topn=3)

今の例では3文しか学習していませんので効果を発揮しませんが、大きなコーパスで学習することで、並列関係のものが近くに来たりなど面白い結果が得られます。


### 可視化
2次元に圧縮することで単語ごとの位置関係を可視化することができます。以下はt-SNEを用いた例です。

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
vocabs = model.wv.vocab.keys()
tsne_model = TSNE(perplexity=40, n_components=2, init="pca", n_iter=5000, random_state=23)
vectors_tsne = tsne_model.fit_transform(model[vocabs])
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(vectors_tsne[:, 0], vectors_tsne[:, 1])
for i, word in enumerate(list(vocabs)):
    plt.annotate(word, xy=(vectors_tsne[i, 0], vectors_tsne[i, 1]))
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()

## 8.IMDB映画レビューデータセットの分散表現

IMDB映画レビューデータセットの訓練データをコーパスとしてWord2Vecを学習させ分散表現を獲得しましょう。


### 【問題5】コーパスの前処理
コーパスの前処理として、特殊文字（!など）やURLの除去、大文字の小文字化といったことを行なってください。また、単語（トークン）はリストで分割してください。

### 5.1.1（解答）

In [ ]:
import re

train_list = []
text_list = []
X_train = x_train.copy()
X_test = x_test.copy()

for i in range(len(X_train)):
    X_train[i] = X_train[i].lower()
    X_train[i] = X_train[i].rstrip()
    X_train[i] = X_train[i].translate(table)
    token = re.split('[, /]',X_train[i])
    train_list.append(token)

    X_test[i] = X_test[i].lower()
    X_test[i] = X_test[i].rstrip()
    X_test[i] = X_test[i].translate(table)
    token1 = re.split('[, /]',X_test[i])
    text_list.append(token1)

In [ ]:
x_train[0]

In [ ]:
train_list[0]

### 【問題6】Word2Vecの学習
Word2Vecの学習を行なってください。

In [ ]:
model = Word2Vec(min_count=1, size=10) # 次元数を10に設定
model.build_vocab(train_list) # 準備
model.train(train_list, total_examples=model.corpus_count, epochs=model.iter) # 学習

In [ ]:
model.wv.most_similar(positive="good", topn=10)

In [ ]:
vocabs = model.wv.vocab.keys()
tsne_model = TSNE(perplexity=40, n_components=2, init="pca", n_iter=5000, random_state=23)
vectors_tsne = tsne_model.fit_transform(model[vocabs])
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(vectors_tsne[:, 0], vectors_tsne[:, 1])
for i, word in enumerate(list(vocabs)):
    plt.annotate(word, xy=(vectors_tsne[i, 0], vectors_tsne[i, 1]))
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()